In [ ]:
from deep_orderbook.recorder import Receiver, Writer
from deep_orderbook.shapper import BookShapper

import asyncio
import pandas as pd
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
MARKETS = ["ETHBTC", "BTCUSDT", "ETHUSDT", "BNBBTC", "BNBETH", "BNBUSDT"]

In [ ]:
#receiver = await Receiver.create(markets=MARKETS)
receiver = await Writer.create(markets=MARKETS, data_folder='../crypto-trading/data')
wrting = asyncio.create_task(receiver.run_writer(save_period_minutes=10))

In [ ]:
b = receiver.depth_managers['ETHBTC'].get_depth_cache().get_bids()
pd.DataFrame(b, columns=['price', 'size']).astype(np.float64).set_index('price')#.cumsum().plot()

In [ ]:
a = receiver.depth_managers['ETHBTC'].get_depth_cache().get_asks()
pd.DataFrame(a, columns=['price', 'size']).astype(np.float64).set_index('price')

In [ ]:
receiver.depth_managers['ETHBTC'].trades

In [ ]:
shappers = {pair: await BookShapper.create() for pair in MARKETS}
multi_replay = receiver.multi_generator(shappers)

In [ ]:
_ = await multi_replay.__anext__()

In [ ]:
import aioitertools
genarr = BookShapper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await aioitertools.next(genarr)

In [ ]:
genacc = BookShapper.accumulate_array(genarr, markets=MARKETS)
_ = await aioitertools.next(genacc)

In [ ]:
async for toshow in BookShapper.images(accumulated_arrays=genacc, every=1, LENGTH=512):
    plt.figure(figsize=(15,11))
    plt.imshow(toshow, origin="upper");
    clear_output(wait=True)
    plt.show()

In [ ]:
#task = asyncio.create_task(images(accumulated_arrays=genacc, every=1, LENGTH=512))